# Erster Korrekturschritt: Verzeichnung

Im ersten Korrekturschritt werden zuerst die intrinsischen Kameraparameter sowie die Verzeichnungsparameter bestimmt. Dafür wird mit dem "Pinhole-Modell" ein Zusammenhang zwischen Weltkoordinaten und Bildkoordinaten hergestellt. 


![Pinhole-Kamera-Modell](media/pinhole.png)
<center>Pinhole-Modell (https://docs.opencv.org/4.5.5/d9/d0c/group__calib3d.html)</center>


Für die Umsetzung werden Schachbrettbilder genutzt, die mit derselben Kamera wie die zu korrigierenden Bilder aufgenommen wurden. Die Schnittpunkte des Schachbrettmusters (Stellen, an denen sich jeweils vier Felder treffen) werden als Weltkoordinaten definiert und im Anschluss auf SubPixel-Niveau im Bild detektiert (Bildkoordinaten).

Die berechneten intrinsischen Kameraparameter und Verzeichungsparameter werden anschließend auf sämtliche Nadir-Bilder angewendet, wodurch die Verzeichnung korrigiert wird.

In [ ]:
import cv2, glob, os
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display

chess_path = r"./chess/" # Pfad der Schachbrettbilder
input_path = r"./input/" # Pfad der Nadir-Rohdaten
output1_path = r"./output1/" # Pfad der primär korrigierten Nadir-Bilder

horizontal_corner_points = int(input("\nAnzahl horizontaler Schachtfelder: "))-1
vertical_corner_points = int(input("Anzahl vertikaler Schachfelder: "))-1
square_size = float(input("Seitenlänge eines Feldes [mm]: "))

# Arrays für Weltkoordinaten (objpoints) und Bildkoordinaten (imgpoints)
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.           

In [ ]:
# Definition der Weltkoordinaten (0,0,0), (1,0,0), (2,0,0)...
objp = np.zeros((vertical_corner_points * horizontal_corner_points, 3), np.float32)
objp[:,:2] = np.mgrid[0:vertical_corner_points, 0:horizontal_corner_points].T.reshape(-1,2)
objp *= square_size

In [ ]:
chess_images = glob.glob(chess_path + "*.jpg")
f = IntProgress(min=0, max=len(chess_images), description='Analyse:',bar_style='warning')
display(f)

print("\nAnalyse der Schachbrettbilder:")

# Einlesen eines jeden JPGs im Schachbrettverzeichnis
for chess_img in chess_images:
    
    # Umwandlung in Graustufe
    gray_img = cv2.imread(chess_img, 0)
    
    #print(chess_img)
                    
    # Detektion der Schnittpunkte im Schachbrettmuster
    ret, corners = cv2.findChessboardCorners(gray_img, (vertical_corner_points, horizontal_corner_points), None)

    if ret == True:
        objpoints.append(objp)

        #criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 0.1)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        # Bestimmung der Schnittpunkte im SubPixel-Bereich
        #corners2 = cv2.cornerSubPix(gray_img,corners,(11,11),(-1,-1),criteria)
        corners2 = cv2.cornerSubPix(gray_img,corners,(5,5),(-1,-1),criteria)
        imgpoints.append(corners2)
        print("%s..." % chess_img,"ok")

    else:
        print("%s ..." % chess_img,"kein Schachbrettmuster gefunden")
    
    f.value += 1

In [ ]:
# Berechnung der intrinsischen und extrinsischen Kameraparameter anhand der Welt- und Bildkoordinaten
if len(imgpoints) != 0 and len(objpoints) != 0:
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray_img.shape[::-1], None, None)
    print("\nVerzeichnungsparameter bestimmt.")

In [ ]:
nadir_images = glob.glob(input_path + "*.jpg")
f = IntProgress(min=0, max=len(nadir_images), description='Korrektur:',bar_style='info')
display(f)

# Einlesen eines jeden Bildes im Input-Verzeichnis
for image in nadir_images:
    
    print("Korrigiere ... ", image)
    img = cv2.imread(image)
    h, w = img.shape[:2]
     
    # Anwenden der berechneten Verzeichnungsparameter auf das eingelesene Bild
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),0,(w,h))
    
    # Korrektur der Verzeichnung
    new_img = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # Bild zuschneiden
    #x,y,w2,h2 = roi
    #newimg = newimg[y:y+h2, x:x+w2]
                                    
    new_img_name = image[8:]
    cv2.imwrite(output1_path + new_img_name, new_img)
    f.value += 1
    
print("\nAlle Bilder korrigiert und im 'output1'-Ordner gespeichert.")

In [ ]:
# Berechnung des durchschnittlichen Fehlers (je näher 0, desto besser)
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print(mean_error)